In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-with-aspect/reliance.csv
/kaggle/input/news-with-aspect/TATA.csv
/kaggle/input/news-with-aspect/apollo.csv


READ THE CSV

In [2]:
df=pd.read_csv("/kaggle/input/news-with-aspect/reliance.csv")

In [3]:
df.head()

,date,news,Predicted Sentiment,Sentiment Probability,aspect_labelling,cleaned_text,syntactic_embeddings,aspects
0,2017-12-18,"So far, Reliance Industries has Hotstar and Ro...",neutral,0.828784,"[[('stake', 'stock_market', 'neutral', 0.92604...",far reliance industry hotstar roy kapur film b...,[-3.25153815e-03 -3.45729113e-01 2.80335128e-...,stake stock_market reliance jio corporate_ne...
1,2017-12-23,The rally in the countrys most valuable compan...,positive,0.914790,"[[('Reliance', 'companies', 'positive', 0.9455...",rally country valuable company reliance indust...,[-5.37841558e-01 -1.09654069e-01 3.00435692e-...,Reliance companies
2,2017-12-24,Mukesh Ambani sets out five aims for Reliance ...,neutral,0.826013,"[[('Reliance', 'companies', 'neutral', 0.58129...",mukesh ambani set five aim reliance industry r...,[ 1.59307327e-02 -2.61955202e-01 4.94948804e-...,Reliance companies mukesh ambani investor_se...
3,2017-12-25,Mukesh Ambani set five aims for the company an...,neutral,0.760601,"[[('mukesh ambani', 'investor_sentiment', 'neu...",mukesh ambani set five aim company said expect...,[ 4.85471934e-02 -3.98840368e-01 3.53910148e-...,mukesh ambani investor_sentiment
4,2018-01-02,The latest deal between the estranged Ambani b...,positive,0.548908,"[[('Reliance', 'companies', 'neutral', 0.62014...",latest deal estranged ambani brother reliance ...,[ 6.33310601e-02 -4.43187535e-01 4.61650789e-...,Reliance companies Reliance companies


In [5]:
import ast

Converting the string of lists to lists 

In [8]:
df["news"]=df["news"].apply(lambda x:list(set(x.split(" | "))))

Calling the huggingface model FINBERT and Tokenizer

In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Model structure

In [11]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
import torch

In [33]:
import torch.nn.functional as F

In [34]:
torch.cuda.set_device(0)

In [35]:
device="cuda:0"

Function to get embedding from the last layer of the model for each sentence passed.

In [36]:
def Embedding(sentence):
    # Disable gradient computation to save memory, as we don't need gradients for inference
    with torch.no_grad():  
        
        # Tokenize the sentence with special tokens, and prepare it for the model as tensor inputs
        # - add_special_tokens=True adds [CLS] and [SEP] tokens for BERT-based models
        # - return_tensors="pt" returns the output as PyTorch tensors
        # - padding="max_length" pads the sentence to the model's max length
        # - truncation=True ensures sentences longer than the max length are truncated
        inputs = tokenizer(sentence, add_special_tokens=True, return_tensors="pt", padding="max_length", truncation=True)
        
        # Pass the tokenized inputs through the model, and request hidden states for embedding extraction
        output = model(**inputs, output_hidden_states=True)
        
        # Extract the last hidden state layer, which represents the sentence embedding
        # Squeeze removes any singleton dimensions (e.g., [1, 768] to [768])
        embedding = output["hidden_states"][-1].squeeze()
        
    # Return only the embedding of the [CLS] token, which captures the overall sentence representation
    return embedding[0]

In [41]:
for i in range(len(df)):
    print(i)  # Print the index to track progress through the DataFrame
    row = df.iloc[i]  # Get the row at index i
    news_embeddings = []  # Initialize a list to store embeddings for each sentence in the news article
    aspect_dict = {}  # Dictionary to store and reuse embeddings for unique aspects
    
    # Loop through each sentence in the news article
    for j in range(len(row["news"])):
        sentence_embedding = Embedding(row["news"][j])  # Get the embedding for the current sentence
        aspects = ast.literal_eval(row["aspect_labelling"])[j]  # Parse aspect labels for the sentence
        
        # If there are no aspects, directly add the sentence embedding to news_embeddings
        if not aspects or aspects == [[]]:
            news_embeddings.append(sentence_embedding)
            continue  # Skip to the next sentence

        aspect_embeddings = []  # Initialize a list to store embeddings for each aspect in the sentence
        
        # Loop through each aspect associated with the sentence
        for aspect, category, sentiment, probs in aspects:
            # Check if the aspect is already in aspect_dict; if not, generate and store its embedding
            if aspect not in aspect_dict:
                aspect_embedding = Embedding(aspect)
                aspect_dict[aspect] = aspect_embedding
            else:
                aspect_embedding = aspect_dict[aspect]
            
            # Calculate attention score between sentence and aspect embeddings using dot product
            attention_scores = torch.matmul(sentence_embedding, aspect_embedding.T)
            attention_weight = F.softmax(attention_scores, dim=-1)  # Apply softmax to normalize attention scores
            
            # Generate a weighted embedding using attention weight and sentiment probability
            weighted_embedding = sentence_embedding * attention_weight * probs  # probs represent sentiment probability
            
            aspect_embeddings.append(weighted_embedding)  # Add the weighted embedding to aspect_embeddings
        
        # If aspect embeddings are present, calculate their mean and add to news_embeddings
        if aspect_embeddings:
            news_embeddings.append(torch.mean(torch.stack(aspect_embeddings), dim=0))
        else:
            # If no aspects were found, add the sentence embedding directly
            news_embeddings.append(sentence_embedding)
    
    # Store the final list of embeddings for each sentence in the current row's "Aspect_embedding" column
    df.at[i, "Aspect_embedding"] = np.array(news_embeddings)


1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569


In [42]:
df.tail()

,date,news,Predicted Sentiment,Sentiment Probability,aspect_labelling,cleaned_text,syntactic_embeddings,aspects,Aspect_embedding
1565,2023-09-01,[Here are the stocks to watch before going int...,neutral,0.939460,"[[('stocks', 'stock_market', 'neutral', 0.9394...",stock watch going trade today,[ 2.01004952e-01 -4.93680447e-01 2.75479019e-...,stocks stock_market,"[[0.27598986, 0.7782642, -0.045324817, -0.2626..."
1566,2023-09-06,"[In July, Jio Financial Services Ltd, the deme...",neutral,0.799973,"[[('Reliance', 'companies', 'neutral', 0.94037...",july jio financial service ltd demerged financ...,[-1.80795610e-01 -2.91528553e-01 3.05293500e-...,Reliance companies,"[[0.57127863, 0.19254339, -0.500566, -0.288088..."
1567,2023-09-08,[Nvidia Corp. plans to partner with the Tata G...,neutral,0.823505,"[[('Reliance', 'companies', 'neutral', 0.68710...",nvidia corp plan partner tata group artificial...,[ 1.60126954e-01 2.47091278e-02 2.74878204e-...,Reliance companies Tata companies mukesh a...,"[[0.26281384, -0.123360805, -0.36010236, -0.14..."
1568,2023-09-09,[Anant Ambani made the donation on behalf of t...,neutral,0.926210,[[]],anant ambani made donation behalf company mukh...,[-1.00057758e-01 -4.06168342e-01 2.28058130e-...,NaN,"[[0.16176419, 0.3424221, -0.4502127, -0.265301..."
1569,2023-09-11,[The U.S.-based chipmaker has tied up with the...,positive,0.728029,[[]],usbased chipmaker tied indian conglomerate bui...,[ 4.12438922e-02 -2.15106636e-01 1.44464299e-...,NaN,"[[0.34263432, 0.010804227, -0.6979692, -0.1366..."


In [59]:
df["date"]

0       2017-12-18
1       2017-12-23
2       2017-12-24
3       2017-12-25
4       2018-01-02
           ...    
1565    2023-09-01
1566    2023-09-06
1567    2023-09-08
1568    2023-09-09
1569    2023-09-11
Name: date, Length: 1570, dtype: object

In [61]:
# Step 1: Convert the date column to datetime format with a four-digit year
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


# Step 2: Sort the data by date (just in case it's unsorted)
df = df.sort_values(by='date').reset_index(drop=True)

# Step 3: Calculate the difference between consecutive dates
df['date_gap'] = df['date'].diff().dt.days  # `.dt.days` extracts the number of days in the difference

# Step 4: Display gaps
print(df)

# Step 5: Summary of gaps
gap_summary = df['date_gap'].value_counts().sort_index()  # Shows how many times each gap occurs
print("Gap summary (in days):\n", gap_summary)

           date                                               news  \
0    2017-12-18  [So far, Reliance Industries has Hotstar and R...   
1    2017-12-23  [The rally in the countrys most valuable compa...   
2    2017-12-24  [Mukesh Ambani sets out five aims for Reliance...   
3    2017-12-25  [Mukesh Ambani set five aims for the company a...   
4    2018-01-02  [The latest deal between the estranged Ambani ...   
...         ...                                                ...   
1565 2023-09-01  [Here are the stocks to watch before going int...   
1566 2023-09-06  [In July, Jio Financial Services Ltd, the deme...   
1567 2023-09-08  [Nvidia Corp. plans to partner with the Tata G...   
1568 2023-09-09  [Anant Ambani made the donation on behalf of t...   
1569 2023-09-11  [The U.S.-based chipmaker has tied up with the...   

     Predicted Sentiment  Sentiment Probability  \
0                neutral               0.828784   
1               positive               0.914790   
2     

For days where news wasn't there,used last known news embeddings

In [63]:
df['date'] = pd.to_datetime(df['date'])

# Step 2: Set 'date' as the index for easier reindexing
df = df.set_index('date')

# Step 3: Create a complete date range from the minimum to maximum dates in the dataset
full_date_range = pd.date_range(start=df.index.min(), end=df.index.max())

# Step 4: Reindex the DataFrame to the complete date range, inserting NaNs where data is missing
df = df.reindex(full_date_range)

# Step 5: Forward-fill NaN values using the last available known value
df = df.fillna(method='ffill')

# Optional: Reset index if you'd like 'date' as a column again
df = df.rename_axis('date').reset_index()

print(df.head())

        date                                               news  \
0 2017-12-18  [So far, Reliance Industries has Hotstar and R...   
1 2017-12-19  [So far, Reliance Industries has Hotstar and R...   
2 2017-12-20  [So far, Reliance Industries has Hotstar and R...   
3 2017-12-21  [So far, Reliance Industries has Hotstar and R...   
4 2017-12-22  [So far, Reliance Industries has Hotstar and R...   

  Predicted Sentiment  Sentiment Probability  \
0             neutral               0.828784   
1             neutral               0.828784   
2             neutral               0.828784   
3             neutral               0.828784   
4             neutral               0.828784   

                                    aspect_labelling  \
0  [[('stake', 'stock_market', 'neutral', 0.92604...   
1  [[('stake', 'stock_market', 'neutral', 0.92604...   
2  [[('stake', 'stock_market', 'neutral', 0.92604...   
3  [[('stake', 'stock_market', 'neutral', 0.92604...   
4  [[('stake', 'stock_market

/tmp/ipykernel_30/3062024545.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


Creating aspect feature for time series model by meaning the embeddings for each day

In [70]:
df["Aspect_feature"]=df["Aspect_embedding"].apply(lambda x: np.mean(x))

In [79]:
df.drop(["date_gap"],axis=1,inplace=True)

In [80]:
df.head()

,date,news,Predicted Sentiment,Sentiment Probability,aspect_labelling,cleaned_text,syntactic_embeddings,aspects,Aspect_embedding,Aspect_feature
0,2017-12-18,"[So far, Reliance Industries has Hotstar and R...",neutral,0.828784,"[[('stake', 'stock_market', 'neutral', 0.92604...",far reliance industry hotstar roy kapur film b...,[-3.25153815e-03 -3.45729113e-01 2.80335128e-...,stake stock_market reliance jio corporate_ne...,"[[0.4806769, 0.2852945, -0.46939108, -0.776523...",-0.017874
1,2017-12-19,"[So far, Reliance Industries has Hotstar and R...",neutral,0.828784,"[[('stake', 'stock_market', 'neutral', 0.92604...",far reliance industry hotstar roy kapur film b...,[-3.25153815e-03 -3.45729113e-01 2.80335128e-...,stake stock_market reliance jio corporate_ne...,"[[0.4806769, 0.2852945, -0.46939108, -0.776523...",-0.017874
2,2017-12-20,"[So far, Reliance Industries has Hotstar and R...",neutral,0.828784,"[[('stake', 'stock_market', 'neutral', 0.92604...",far reliance industry hotstar roy kapur film b...,[-3.25153815e-03 -3.45729113e-01 2.80335128e-...,stake stock_market reliance jio corporate_ne...,"[[0.4806769, 0.2852945, -0.46939108, -0.776523...",-0.017874
3,2017-12-21,"[So far, Reliance Industries has Hotstar and R...",neutral,0.828784,"[[('stake', 'stock_market', 'neutral', 0.92604...",far reliance industry hotstar roy kapur film b...,[-3.25153815e-03 -3.45729113e-01 2.80335128e-...,stake stock_market reliance jio corporate_ne...,"[[0.4806769, 0.2852945, -0.46939108, -0.776523...",-0.017874
4,2017-12-22,"[So far, Reliance Industries has Hotstar and R...",neutral,0.828784,"[[('stake', 'stock_market', 'neutral', 0.92604...",far reliance industry hotstar roy kapur film b...,[-3.25153815e-03 -3.45729113e-01 2.80335128e-...,stake stock_market reliance jio corporate_ne...,"[[0.4806769, 0.2852945, -0.46939108, -0.776523...",-0.017874


In [82]:
df.to_csv("/kaggle/working/updated_aspect_feature.csv")